# **Data Preprocessing**

This notebook describes the entire process of **preprocessing MODIS and ERA5 Land (NetCDF) data** and preparing it for **machine learning model training**. The main objective is to generate a dataset where each row represents a specific region and year, with monthly aggregated values of relevant environmental variables.

In [1]:
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.mask import mask
from rasterio.transform import rowcol
from rasterio.warp import transform

import geopandas as gpd
from shapely.geometry import box
import xarray as xr

from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed

## **MODIS Data Preprocessing and Preparation for Model Training**

This notebook outlines the process of **preprocessing MODIS data** and **preparing it for machine learning model training**. The process is divided into three main steps:

1. **Generating Threshold-Filtered TIF Files**  
2. **Extracting Sample Points from CropMap**  
3. **Extracting Multi Values to Points**

The goal is to create a dataset where each region contains monthly weighted average values of various MODIS indicators (e.g., NDVI, EVI, LST) for use in crop yield estimation and other analyses.

### **Step 1: Generating Threshold-Filtered TIF Files**

The first step involves generating **threshold-filtered TIF files** by counting the number of crop pixels within each MODIS grid cell. A new TIF file is created where each pixel value represents the number of crop pixels in the corresponding MODIS grid cell.

#### **Steps**:
1. Load the CropMap data and MODIS grid information.
2. Count the number of crop pixels within each MODIS grid cell for different resolutions (250m, 500m, 1000m).
3. Create a new TIF file where each pixel value represents the number of crop pixels.

#### **Result**:
- A set of **threshold-filtered TIF files** for different MODIS resolutions (250m, 500m, 1000m), which will be used in the next step.

In [ ]:
def count_crop_pixels_in_grid(modis_path, cropmap_path, crop_type, output_path, grid_size=1000):
    # Read MODIS transform and dimensions
    with rasterio.open(modis_path) as modis_src:
        modis_transform = modis_src.transform
        modis_crs = modis_src.crs
        modis_width = modis_src.width
        modis_height = modis_src.height

    # Read CropMap data
    with rasterio.open(cropmap_path) as cropmap_src:
        cropmap_data = cropmap_src.read(1)
        cropmap_transform = cropmap_src.transform
        cropmap_crs = cropmap_src.crs

    # Ensure CRS match
    if cropmap_crs != modis_crs:
        raise ValueError("CRS of CropMap and MODIS data do not match.")
    
    # Calculate the new dimensions for 250m grid based on MODIS dimensions
    new_width = modis_width
    new_height = modis_height
    
    # Create an array to store the counts of crop_type pixels
    grid_counts = np.zeros((new_height, new_width), dtype=np.uint16)
    
    # Iterate over the MODIS grid with tqdm for progress tracking
    for i in tqdm(range(new_height), desc="Processing rows"):
        for j in range(new_width):
            # Calculate the coordinates of the MODIS pixel
            modis_x, modis_y = modis_transform * (j, i)
            
            # Find the corresponding window in the CropMap
            col_off, row_off = ~cropmap_transform * (modis_x, modis_y)
            col_off = int(col_off)
            row_off = int(row_off)
            
            # Define the window size in CropMap pixels
            window_size = grid_size // 10
            window = cropmap_data[
                row_off:row_off + window_size,
                col_off:col_off + window_size
            ]
            
            # Count the number of pixels with the specified crop_type
            grid_counts[i, j] = np.sum(window == crop_type)
    
    # Define the new transform for the 250m resolution based on MODIS transform
    new_transform = modis_transform
    
    # Write the new raster file with 250m resolution
    with rasterio.open(
        output_path, 'w', driver='GTiff', height=new_height,
        width=new_width, count=1, dtype=grid_counts.dtype,
        crs=modis_crs, transform=new_transform
    ) as dst:
        dst.write(grid_counts, 1)


# Paths to your raster files
modis_path = '/home/sehoon/Desktop/Sehoon/crop_yield/data/MODIS/LST/MODIS_LST_Ukraine_2010-03-22_2.tif'
cropmap_path = '/home/sehoon/Desktop/Sehoon/crop_yield/data/EU_CropMap_22_v1_stratum_UA-HR.tif'
crop_type = 211
output_path = '/home/sehoon/Desktop/Sehoon/crop_yield/data/cropmap/wheat/1000m.tif'

count_crop_pixels_in_grid(modis_path, cropmap_path, crop_type, output_path)

### **Step 2: Extracting Sample Points from CropMap**

In this step, sample points are extracted from the **threshold-filtered TIF files** generated in Step 1 based on specific regions and resolutions. Each sample point includes the following information:

- **`crop_ratio`**: The proportion of the MODIS pixel covered by the crop.
- **`overlap_ratio`**: The ratio of the MODIS pixel area that overlaps with the region polygon.

#### **Steps**:
1. Load the threshold-filtered TIF files and region polygons (GeoJSON format).
2. For each region, extract the MODIS grid cells that overlap with the region.
3. Calculate `crop_ratio` and `overlap_ratio` for each grid cell.
4. Save the extracted sample points as CSV files for each region.

#### **Result**:
- CSV files containing **sample points with `crop_ratio` and `overlap_ratio`** for each region.


In [ ]:
def calculate_overlap(region_polygon, pixel_geometry):
    """Calculate the overlap ratio between a pixel and the region polygon."""
    intersection_area = region_polygon.intersection(pixel_geometry).area
    pixel_area = pixel_geometry.area
    return intersection_area / pixel_area

def process_region_with_overlap(region, land_cover_map, output_dir, resolution):
    region_geometry = [region['geometry']]
    region_name = region['name']
    
    with rasterio.open(land_cover_map) as src:
        try:
            out_image, out_transform = mask(src, region_geometry, crop=False)
        except ValueError:
            return f"Region {region_name} does not overlap with the land cover map."

    # Consider all valid pixels (non-masked)
    rows, cols = np.where(out_image[0] > 0)
    
    if rows.size == 0:
        return f"No overlapping pixels found for region {region_name}."

    crop_ratios = []
    overlap_ratios = []

    for row, col in zip(rows, cols):
        pixel_value = out_image[0, row, col]
        crop_ratio = pixel_value / ((resolution / 10) ** 2)
        
        # Get the four corner coordinates of the pixel
        x_min, y_max = rasterio.transform.xy(out_transform, row, col, offset='ul')
        x_max, y_min = rasterio.transform.xy(out_transform, row, col, offset='lr')
        
        # Create a box representing the pixel
        pixel_box = box(x_min, y_min, x_max, y_max)
        
        overlap_ratio = calculate_overlap(region_geometry[0], pixel_box)
        
        crop_ratios.append(crop_ratio)
        overlap_ratios.append(overlap_ratio)

    df = pd.DataFrame({
        'idx_x': rows,
        'idx_y': cols,
        'crop_ratio': crop_ratios,
        'overlap_ratio': overlap_ratios
    })

    safe_region_name = "".join([c if c.isalnum() else "_" for c in region_name])
    output_file = os.path.join(output_dir, f"{safe_region_name}.csv")
    df.to_csv(output_file, index=False)

    return f"Saved region {region_name} to {output_file}"

def extract_sample_points_by_region_with_overlap(land_cover_map, region_file, output_dir, resolution, num_cpus=1):
    with rasterio.open(land_cover_map) as src:
        crs = src.crs

    regions = gpd.read_file(region_file)

    if regions.crs != crs:
        regions = regions.to_crs(crs)

    tasks = [
        (region, land_cover_map, output_dir, resolution)
        for _, region in regions.iterrows()
    ]

    results = []
    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        future_to_region = {executor.submit(process_region_with_overlap, *task): task[0]['name'] for task in tasks}
        for future in tqdm(as_completed(future_to_region), total=len(future_to_region), desc="Processing regions"):
            region_name = future_to_region[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as exc:
                results.append(f'{region_name} generated an exception: {exc}')

    for result in results:
        print(result)

resolution = 1000

# Example usage
land_cover_map = f'/home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/{resolution}m/{resolution}m.tif'
region_file = '/home/sehoon/Desktop/Sehoon/crop_yield/data/ua.json'
output_dir = f'/home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/{resolution}m'
num_cpus = 12

extract_sample_points_by_region_with_overlap(land_cover_map, region_file, output_dir, resolution, num_cpus=num_cpus)

### **Step 3: Extracting Multi Values to Points**

The final step involves extracting **MODIS indicator values** for the sample points and calculating the **monthly weighted average values** using the `crop_ratio` and `overlap_ratio` as weights.

#### **Steps**:
1. Load the sample points (CSV files) and apply thresholds to filter the data.
2. For each MODIS file (NDVI, EVI, LST, etc.), extract the values corresponding to the sample points.
3. Calculate the weighted average of the MODIS values using `crop_ratio * overlap_ratio` as weights.
4. Save the final results as yearly CSV files, where each file contains the monthly weighted average values for each region.

#### **Result**:
- Yearly CSV files with **monthly weighted average MODIS indicator values** (`NDVI`, `EVI`, `LST_Day`, `LST_Night`, etc.) for each region.


In [4]:
def extract_and_sum_modis_values_single_csv(modis_file, sample_csv_path, summary_key, crop_threshold=0.0, overlap_threshold=0.0):
    # Extract year and month from file path
    year, month = os.path.basename(os.path.dirname(os.path.dirname(modis_file))), os.path.basename(os.path.dirname(modis_file))
    sample_df = pd.read_csv(sample_csv_path)

    # Apply thresholds to filter the data
    filtered_df = sample_df[(sample_df['crop_ratio'] >= crop_threshold) & (sample_df['overlap_ratio'] >= overlap_threshold)]

    # Check if there are valid points after filtering
    if filtered_df.empty:
        # print(f"No valid points found for {os.path.basename(sample_csv_path)} after applying thresholds.")
        return None

    idx_x = filtered_df['idx_x'].values
    idx_y = filtered_df['idx_y'].values
    crop_ratios = filtered_df['crop_ratio'].values
    overlap_ratios = filtered_df['overlap_ratio'].values
    combined_weights = crop_ratios * overlap_ratios

    region_name = os.path.splitext(os.path.basename(sample_csv_path))[0]
    summary = {"region_name": region_name, "year": int(year), "month": int(month), summary_key: 0.0}

    with rasterio.open(modis_file) as src:
        # Read the values from the MODIS file based on the indices
        values = src.read(1, masked=True)[idx_x, idx_y]

        # Scale the values according to the summary key
        if summary_key in ["NDVI", "EVI"]:
            values = values / 12000.0
        elif summary_key in ["LST_Day", "LST_Night"]:
            values = values * 0.02
        elif summary_key in ["LAI", "FPAR"]:
            values = values / 255.0

        # Calculate the weighted average using combined weights
        summary[summary_key] = np.sum(values * combined_weights) / np.sum(combined_weights)

    return pd.DataFrame([summary])

def process_year_month(year, month, base_modis_dir, csv_folder, crop_threshold=0.0, overlap_threshold=0.0):
    modis_dir = os.path.join(base_modis_dir, str(year), f"{month:02d}")
    if not os.path.exists(modis_dir):
        print(f"MODIS directory {modis_dir} does not exist.")
        return None

    # Prepare a list of all possible indicators
    indicators = ["LST_Night", "NDVI", "EVI", "LAI", "FPAR", "LST_Day"]
    all_results = []

    for modis_file in os.listdir(modis_dir):
        modis_path = os.path.join(modis_dir, modis_file)
        if not modis_file.endswith('.tif'):
            continue

        if "NDVI" in modis_file:
            sample_points_folder = os.path.join(csv_folder, '250m')
            summary_key = "NDVI"
        elif "EVI" in modis_file:
            sample_points_folder = os.path.join(csv_folder, '250m')
            summary_key = "EVI"
        elif "LAI" in modis_file:
            sample_points_folder = os.path.join(csv_folder, '500m')
            summary_key = "LAI"
        elif "FPAR" in modis_file:
            sample_points_folder = os.path.join(csv_folder, '500m')
            summary_key = "FPAR"
        elif "LST_Day" in modis_file:
            sample_points_folder = os.path.join(csv_folder, '1000m')
            summary_key = "LST_Day"
        elif "LST_Night" in modis_file:
            sample_points_folder = os.path.join(csv_folder, '1000m')
            summary_key = "LST_Night"
        else:
            print(f"Skipping unknown MODIS file: {modis_file}")
            continue

        csv_files = [os.path.join(sample_points_folder, f) for f in os.listdir(sample_points_folder) if f.endswith('.csv')]

        for csv_file in csv_files:
            result_df = extract_and_sum_modis_values_single_csv(modis_path, csv_file, summary_key, crop_threshold, overlap_threshold)
            if result_df is not None:
                all_results.append(result_df)

    if all_results:
        # Combine all results into a single DataFrame
        combined_df = pd.concat(all_results, axis=0).groupby(['region_name', 'year', 'month'], as_index=False).mean()

        # Ensure all indicators are present in the final DataFrame
        for indicator in indicators:
            if indicator not in combined_df.columns:
                combined_df[indicator] = np.nan

        # print(f"Processed {year}-{month:02d} DataFrame:")
        # print(combined_df.head())  # 첫 몇 줄 출력
        return combined_df
    return None

def main_parallel(csv_folder, base_modis_dir, output_folder, num_cpus=8, crop_threshold=0.0, overlap_threshold=0.0):
    os.makedirs(output_folder, exist_ok=True)
    
    for year in range(2010, 2024):
        year_dfs = []

        tasks = [(year, month, base_modis_dir, csv_folder, crop_threshold, overlap_threshold) for month in range(3, 11)]
        with ProcessPoolExecutor(max_workers=num_cpus) as executor:
            futures = [executor.submit(process_year_month, *task) for task in tasks]
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Processing year {year}"):
                result_df = future.result()
                if result_df is not None:
                    year_dfs.append(result_df)

        if year_dfs:
            final_df = pd.concat(year_dfs, ignore_index=True)
            print(f"Final DataFrame for year {year}:")
            print(final_df.head())  # 연도별 최종 데이터프레임의 첫 몇 줄 출력
            final_df.to_csv(os.path.join(output_folder, f'results_{year}.csv'), index=False, na_rep='')

# Example usage
csv_folder = '/home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat'
base_modis_dir = '/home/sehoon/Desktop/Sehoon/crop_yield/data/MODIS/preprocessed'
output_folder = '/home/sehoon/Desktop/Sehoon/crop_yield/wheat/MODIS'

main_parallel(csv_folder, base_modis_dir, output_folder, num_cpus=12, crop_threshold=0.7, overlap_threshold=0.7)

Processing year 2010: 100%|██████████| 8/8 [00:21<00:00,  2.74s/it]

Final DataFrame for year 2010:
                 region_name  year  month   LST_Night       LAI       EVI  \
0  Avtonomna_Respublika_Krym  2010      4  277.020871  0.054268  0.295739   
1                  Cherkaska  2010      4  277.354322  0.022917  0.206472   
2               Chernihivska  2010      4  276.693037  0.019556  0.204409   
3               Chernivetska  2010      4  272.585667  0.040966  0.274846   
4            Dnipropetrovska  2010      4  277.310531  0.021609  0.201053   

       FPAR     LST_Day      NDVI  
0  0.191749  288.390788  0.472691  
1  0.106383  297.213135  0.376571  
2  0.098242  293.648550  0.362573  
3  0.164644  296.100335  0.461532  
4  0.109640  296.909267  0.398972  



Processing year 2011: 100%|██████████| 8/8 [00:22<00:00,  2.77s/it]

Final DataFrame for year 2011:
                 region_name  year  month      NDVI       EVI      FPAR  \
0  Avtonomna_Respublika_Krym  2011      7  0.284149  0.174652  0.118545   
1                  Cherkaska  2011      7  0.572646  0.400503  0.257863   
2               Chernihivska  2011      7  0.584747  0.382728  0.249903   
3               Chernivetska  2011      7  0.557559  0.393830  0.250263   
4            Dnipropetrovska  2011      7  0.513674  0.348113  0.219681   

    LST_Night       LAI     LST_Day  
0  291.940660  0.027370  311.731868  
1  289.833573  0.101425  301.494555  
2  284.324792  0.092672  298.570866  
3  288.560554  0.093034  301.267216  
4  291.165493  0.059155  305.046409  



Processing year 2012: 100%|██████████| 8/8 [00:22<00:00,  2.77s/it]

...

Final DataFrame for year 2021:
                 region_name  year  month      NDVI       EVI       LAI  \
0  Avtonomna_Respublika_Krym  2021     10  0.247953  0.109530  0.009082   
1                  Cherkaska  2021     10  0.299400  0.132353  0.011624   
2               Chernihivska  2021     10  0.331868  0.162353  0.015800   
3               Chernivetska  2021     10  0.267653  0.125817  0.013273   
4            Dnipropetrovska  2021     10  0.294104  0.128196  0.009231   

      LST_Day      FPAR   LST_Night  
0  292.400670  0.070888  277.422827  
1  278.464218  0.087205  273.678390  
2  284.709661  0.107673  272.975458  
3  288.900956  0.102531  273.960574  
4  280.973778  0.076316  266.719155  



Processing year 2022: 100%|██████████| 8/8 [00:21<00:00,  2.74s/it]

Final DataFrame for year 2022:
                 region_name  year  month   LST_Night     LST_Day       EVI  \
0  Avtonomna_Respublika_Krym  2022      4  272.241934  289.991880  0.374076   
1                  Cherkaska  2022      4  273.523801  292.947132  0.363869   
2               Chernihivska  2022      4  218.314267  271.149041  0.346468   
3               Chernivetska  2022      4  264.153967  299.075694  0.292176   
4            Dnipropetrovska  2022      4  277.835009  264.816869  0.339889   

       NDVI       LAI      FPAR  
0  0.551498  0.061489  0.208872  
1  0.564801  0.046638  0.169256  
2  0.542309  0.044575  0.161627  
3  0.485795  0.038295  0.163286  
4  0.555445  0.035742  0.149019  



Processing year 2023: 100%|██████████| 8/8 [00:22<00:00,  2.77s/it]

Final DataFrame for year 2023:
                 region_name  year  month       LAI     LST_Day      NDVI  \
0  Avtonomna_Respublika_Krym  2023      5  0.046367  296.347554  0.442948   
1                  Cherkaska  2023      5  0.052559  303.894444  0.491472   
2               Chernihivska  2023      5  0.068936  302.150935  0.495381   
3               Chernivetska  2023      5  0.068123  299.615820  0.521383   
4            Dnipropetrovska  2023      5  0.034126  302.181101  0.466210   

        EVI      FPAR   LST_Night  
0  0.283405  0.154004  280.847640  
1  0.329606  0.155615  283.619810  
2  0.353171  0.204766  282.078831  
3  0.381098  0.180909  282.533250  
4  0.283206  0.136105  283.535549  


## **NetCDF Data Preprocessing and Preparation for Model Training**

This notebook outlines the process of **preprocessing NetCDF data** and **preparing it for machine learning model training**. The process is divided into two main steps:

1. **Extracting Sample Points from CropMap for NetCDF**  
2. **Extracting and Averaging NetCDF Values**

The goal is to create a dataset where each region contains monthly weighted average values of various NetCDF variables (e.g., temperature, precipitation) for use in crop yield estimation and other analyses.

### **Step 1: Extracting Sample Points from CropMap for NetCDF**

The first step involves extracting **sample points from CropMap data** based on NetCDF grid cells and regions. Each sample point contains information about the proportion of crop pixels (`crop_ratio`) and the overlap ratio (`overlap_ratio`) between the grid cell and the region polygon.

#### **Steps**:
1. Load the **NetCDF grid information** (longitude, latitude, resolution).
2. Load the **CropMap data** and region polygons (GeoJSON format).
3. For each NetCDF grid cell, calculate:
   - **`crop_ratio`**: Proportion of the grid cell containing the specified crop.
   - **`overlap_ratio`**: Proportion of the grid cell that overlaps with the region polygon.
4. Save the extracted sample points as **CSV files**, where each file corresponds to a specific region.

#### **Result**:
- A set of **CSV files** containing sample points with `crop_ratio` and `overlap_ratio` for each region.

In [7]:
def analyze_tif_with_netcdf(netcdf_info, tif_path, target_value=211):
    crs = netcdf_info['crs']
    lon_min = netcdf_info['lon_min']
    lon_max = netcdf_info['lon_max']
    lat_min = netcdf_info['lat_min']
    lat_max = netcdf_info['lat_max']
    lon_res = netcdf_info['lon_res']
    lat_res = netcdf_info['lat_res']

    results = []

    with rasterio.open(tif_path) as tif:
        tif_crs = tif.crs

        for lat_index, lat in enumerate(np.arange(lat_min, lat_max, lat_res)):
            for lon_index, lon in enumerate(np.arange(lon_min, lon_max, lon_res)):
                lon_start, lon_end = lon, lon + lon_res
                lat_start, lat_end = lat, lat + lat_res

                if tif_crs != crs:
                    lon_start, lat_start = transform(crs, tif_crs, [lon_start], [lat_start])
                    lon_end, lat_end = transform(crs, tif_crs, [lon_end], [lat_end])
                    lon_start, lon_end = lon_start[0], lon_end[0]
                    lat_start, lat_end = lat_start[0], lat_end[0]

                row_start, col_start = rowcol(tif.transform, lon_start, lat_end)
                row_stop, col_stop = rowcol(tif.transform, lon_end, lat_start)

                row_start = max(0, row_start)
                col_start = max(0, col_start)
                row_stop = min(tif.height, row_stop)
                col_stop = min(tif.width, col_stop)

                if row_stop > row_start and col_stop > col_start:
                    window_data = tif.read(1, window=((row_start, row_stop), (col_start, col_stop)))
                    target_count = np.sum(window_data == target_value)
                    total_count = window_data.size
                    ratio = target_count / total_count

                    results.append({
                        'lat_index': lat_index,
                        'lon_index': lon_index,
                        'ratio': ratio,
                        'geom': box(lon_start, lat_start, lon_end, lat_end)
                    })

    return results

def calculate_polygon_overlap(region, netcdf_results):
    points = []
    
    for result in netcdf_results:
        netcdf_pixel = gpd.GeoDataFrame([result], geometry=[result['geom']], crs=region.crs)
        intersection = gpd.overlay(netcdf_pixel, region, how='intersection')
        
        for _, intersected_row in intersection.iterrows():
            overlap_ratio = intersected_row['geometry'].area / netcdf_pixel.geometry.area.iloc[0]
            points.append({
                'idx_x': result['lat_index'],
                'idx_y': result['lon_index'],
                'crop_ratio': result['ratio'],
                'overlap_ratio': overlap_ratio
            })
    
    return points

def process_region(region, netcdf_results, tif_crs, output_dir):
    region_name = region['name']
    region_gdf = gpd.GeoDataFrame([region], geometry='geometry', crs=tif_crs)

    # Calculate the overlap between netcdf pixels and the current region
    sample_points = calculate_polygon_overlap(region_gdf, netcdf_results)

    # Save to CSV with the region name
    if sample_points:
        sample_points_df = pd.DataFrame(sample_points)
        safe_region_name = "".join([c if c.isalnum() else "_" for c in region_name])
        output_file = os.path.join(output_dir, f"{safe_region_name}.csv")
        sample_points_df.to_csv(output_file, index=False)
        return f"Saved region {region_name} to {output_file}"
    else:
        return f"No overlapping pixels found for region {region_name}"

def create_sample_points(netcdf_info, tif_path, region_file, target_value=211, output_dir='output', max_workers=4):
    netcdf_results = analyze_tif_with_netcdf(netcdf_info, tif_path, target_value)
    regions = gpd.read_file(region_file)

    # TIF 파일의 CRS 확인
    with rasterio.open(tif_path) as tif:
        tif_crs = tif.crs

    # 폴리곤 파일이 TIF 파일의 CRS와 다르면 변환
    if regions.crs != tif_crs:
        print(f"Transforming region CRS from {regions.crs} to {tif_crs}")
        regions = regions.to_crs(tif_crs)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 병렬 처리를 위해 ProcessPoolExecutor 사용
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_region, region, netcdf_results, tif_crs, output_dir): region['name']
            for _, region in regions.iterrows()
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing regions"):
            region_name = futures[future]
            try:
                result = future.result()
                print(result)
            except Exception as e:
                print(f"Region {region_name} generated an exception: {e}")

# 예제 사용
netcdf_info = {
    'crs': 'EPSG:4326',
    'lon_min': 22.0,
    'lon_max': 40.5,
    'lat_min': 44.0,
    'lat_max': 52.5,
    'lon_res': 0.10000000149011612,
    'lat_res': 0.10000000149011612,
    'transform': [[0.10, 0.00, 22.00], [0.00,-0.10, 52.50], [0.00, 0.00, 1.00]]
}

tif_path = '/home/sehoon/Desktop/Sehoon/crop_yield/data/EU_CropMap_22_v1_stratum_UA-HR.tif'
region_file = '/home/sehoon/Desktop/Sehoon/crop_yield/data/ua.json'
output_dir = '/home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land'

create_sample_points(netcdf_info, tif_path, region_file, target_value=211, output_dir=output_dir, max_workers=12)

Transforming region CRS from EPSG:4326 to EPSG:3035


Processing regions:   4%|▎         | 1/27 [00:56<24:20, 56.17s/it]

Saved region Ivano-Frankivska to /home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land/Ivano_Frankivska.csv


Processing regions:   7%|▋         | 2/27 [00:56<09:46, 23.44s/it]

Saved region Zakarpatska to /home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land/Zakarpatska.csv


...

Saved region Sevastopilska to /home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land/Sevastopilska.csv


Processing regions:  96%|█████████▋| 26/27 [02:46<00:09,  9.06s/it]

Saved region Kyivska to /home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land/Kyivska.csv


Processing regions: 100%|██████████| 27/27 [02:47<00:00,  6.20s/it]

Saved region Chernihivska to /home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land/Chernihivska.csv


### **Step 2: Extracting and Averaging NetCDF Values**

The second step involves using the **sample points extracted in Step 1** to retrieve values from the NetCDF files and calculate the **monthly weighted average values** using the `crop_ratio` and `overlap_ratio` as weights.

#### **Steps**:
1. Load the sample points (CSV files) and apply thresholds to filter the data based on `crop_ratio` and `overlap_ratio`.
2. Load the **NetCDF data** and extract the values corresponding to the sample points for each time step.
3. Calculate the **weighted average** for each variable using the product of `crop_ratio` and `overlap_ratio` as weights.
4. Save the final results as **yearly CSV files**, where each file contains the monthly weighted average values for each region.

#### **Result**:
- Yearly CSV files with **monthly weighted average NetCDF variable values** (e.g., temperature, precipitation) for each region.


In [14]:
def filter_and_get_indices(csv_path, min_crop_ratio=0.1, max_overlap_ratio=0.7):
    df = pd.read_csv(csv_path)

    # `crop_ratio`가 min_crop_ratio 이상이고 `overlap_ratio`가 max_overlap_ratio 이하인 샘플만 선택
    df = df[(df['crop_ratio'] >= min_crop_ratio) & (df['overlap_ratio'] <= max_overlap_ratio)]
    
    if df.empty:
        return None, None, None

    idx_x = df['idx_x'].values
    idx_y = df['idx_y'].values
    crop_ratios = df['crop_ratio'].values
    overlap_ratios = df['overlap_ratio'].values

    # 최종 가중치 계산 (crop_ratio * overlap_ratio)
    final_weights = crop_ratios * overlap_ratios
    
    return idx_x, idx_y, final_weights

def extract_and_average_pixel_values(netcdf_path, lat_indices, lon_indices, weights):
    ds = xr.open_dataset(netcdf_path)
    data_vars = {var: ds[var].values for var in ds.data_vars}
    
    results = {var: [] for var in ds.data_vars}
    
    time_steps = ds.sizes['time']
    
    for time_step in range(time_steps):
        sum_values = {var: 0 for var in ds.data_vars}
        valid_count = {var: 0 for var in ds.data_vars}
        
        for var, data in data_vars.items():
            values = data[time_step, lat_indices, lon_indices]
            valid_mask = ~np.isnan(values)
            weighted_sum = np.sum(values[valid_mask] * weights[valid_mask])
            valid_count[var] = np.sum(valid_mask)
            sum_values[var] += weighted_sum
        
        avg_values = {var: (sum_values[var] / np.sum(weights[valid_mask]) if valid_count[var] > 0 else np.nan) for var in ds.data_vars}
        
        for var in ds.data_vars:
            results[var].append(avg_values[var])
    
    avg_df = pd.DataFrame(results, index=ds['time'].values)
    
    return avg_df

def process_csv_file(csv_file, netcdf_path, output_folder):
    lat_indices, lon_indices, weights = filter_and_get_indices(csv_file)
    
    if lat_indices is None:
        return f"Skipping {csv_file} - no valid points with crop_ratio >= 0.03 and overlap_ratio <= 0.7"

    avg_df = extract_and_average_pixel_values(netcdf_path, lat_indices, lon_indices, weights)
    
    output_csv_path = os.path.join(output_folder, f'avg_{os.path.basename(csv_file)}')
    avg_df.to_csv(output_csv_path)
    return f"Saved results to {output_csv_path}"

def process_all_csv_files_in_folder(folder_path, netcdf_path, output_folder, num_cpus=12):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    tasks = [(csv_file, netcdf_path, output_folder) for csv_file in csv_files]
    results = []
    
    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        future_to_csv = {executor.submit(process_csv_file, *task): task[0] for task in tasks}
        for future in tqdm(as_completed(future_to_csv), total=len(future_to_csv), desc='Processing CSV files'):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"Exception for {future_to_csv[future]}: {e}")
    
    for result in results:
        print(result)

# Example usage
folder_path = '/home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land'
netcdf_path = '/home/sehoon/Desktop/Sehoon/crop_yield/data/ERA5_Land.nc'
output_folder = '/home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land'

process_all_csv_files_in_folder(folder_path, netcdf_path, output_folder, num_cpus=12)

Processing CSV files: 100%|██████████| 26/26 [00:00<00:00, 44.19it/s]

Skipping /home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land/Sevastopilska.csv - no valid points with crop_ratio >= 0.03 and overlap_ratio <= 0.7
Skipping /home/sehoon/Desktop/Sehoon/crop_yield/sample_points/wheat/ERA5_Land/Kyivska.csv - no valid points with crop_ratio >= 0.03 and overlap_ratio <= 0.7
Saved results to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land/avg_Luhanska.csv
Saved results to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land/avg_Kirovohradska.csv
Saved results to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land/avg_Chernihivska.csv
Saved results to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land/avg_Lvivska.csv
Saved results to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land/avg_Zhytomyrska.csv
Saved results to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land/avg_Vinnytska.csv
Saved results to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land/avg_Ternopilska.csv
Saved results to /home/sehoon/Desktop/Seho

## **Final Step: Combining MODIS and ERA5 Land Data**

The final step involves **combining the preprocessed MODIS and ERA5 Land data** into a single dataset. This merged dataset will be used for further analysis and model training.

### **Output Example**:
| **region_name** | **year** | **month** | **t2m** | **stl1** | **stl2** | **ssr** | **tp** | **swvl1** | **swvl2** | **NDVI** | **EVI** | **LST_Day** | **LST_Night** |
|-----------------|----------|-----------|---------|----------|----------|---------|-------|----------|----------|----------|---------|-------------|---------------|
| Ivano_Frankivska         | 2013     | 3         | 2.5     | 1.8      | 1.5      | 450.6   | 0.12  | 0.35     | 0.25     | 0.71     | 0.65    | 300.5       | 290.1         |
| Luhanska         | 2013     | 4         | 5.2     | 3.1      | 2.7      | 500.2   | 0.18  | 0.42     | 0.30     | 0.74     | 0.69    | 302.0       | 292.8         |

In [15]:
def process_era5_land_csv_file(file_path):
    # ERA5 Land 파일 처리
    region_name = os.path.splitext(os.path.basename(file_path))[0].split('_')[1]
    
    df = pd.read_csv(file_path)
    
    # Convert the date to year and month
    df['date'] = pd.to_datetime(df['Unnamed: 0'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    
    # Rearrange columns and rename them as needed
    new_df = df[['year', 'month', 't2m', 'stl1', 'stl2', 'ssr', 'tp', 'swvl1', 'swvl2']]
    new_df.insert(0, 'region_name', region_name)
    
    return new_df

def combine_csv_files(folder_path, is_era5_land=True):
    all_data = pd.DataFrame()
    
    # Iterate over all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            if is_era5_land:
                df = process_era5_land_csv_file(file_path)
            else:
                df = pd.read_csv(file_path)  # 일반 CSV 파일 읽기
            
            all_data = pd.concat([all_data, df], ignore_index=True)
    
    return all_data

def process_and_merge_data(era5_land_folder, modis_folder, output_file):
    # ERA5 Land 데이터 처리 및 병합
    era5_land_data = combine_csv_files(era5_land_folder, is_era5_land=True)
    
    # MODIS 데이터 처리 및 병합
    modis_data = combine_csv_files(modis_folder, is_era5_land=False)
    
    # 두 데이터프레임 병합
    combined_df = pd.merge(era5_land_data, modis_data, on=['region_name', 'year', 'month'])
    
    # 병합된 데이터 저장
    combined_df.to_csv(output_file, index=False)
    print(f"Combined data saved to {output_file}")

# Example usage
era5_land_folder = '/home/sehoon/Desktop/Sehoon/crop_yield/wheat/ERA5_Land'  # ERA5 Land CSV 파일들이 있는 폴더 경로
modis_folder = '/home/sehoon/Desktop/Sehoon/crop_yield/wheat/MODIS'  # MODIS CSV 파일들이 있는 폴더 경로
output_file = '/home/sehoon/Desktop/Sehoon/crop_yield/wheat/combined_data.csv'

process_and_merge_data(era5_land_folder, modis_folder, output_file)

Combined data saved to /home/sehoon/Desktop/Sehoon/crop_yield/wheat/combined_data.csv
